In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import tensorflow as tf
import numpy as np
import os

Using TensorFlow backend.


In [2]:
# load in the amazon data 
directory = '.\\domain_adaptation_images\\amazon\\images'
amazon_data = []
amazon_labels = []
class_directories = [x[0] for x in os.walk(directory)]
for i in range(len(class_directories) - 1):
    sub_directory = class_directories[i+1]
    # get all the file names & path
    files = [f for f in os.listdir(sub_directory)]
    file_path = [sub_directory + '\\' + file for file in files]
    # add into the dataset and label 
    for f_p in file_path:
        img = image.load_img(f_p, target_size = (224, 224))
        x = image.img_to_array(img)
        #x = np.extend_dims(x, axis = 0)
        x = preprocess_input(x)
        amazon_data.append(x)
        amazon_labels.append(i + 1)

In [15]:
# convert it to numpy array
amazon_data = np.asarray(amazon_data)
amazon_labels = np.asarray(amazon_labels)

# training data preparation
n_data = amazon_data.shape[0]
n_classes = 31
random_index = np.random.permutation(n_data)
amazon_training_data = amazon_data[random_index]
amazon_training_labels = amazon_labels[random_index]
print (amazon_training_data.shape, amazon_training_labels.shape)

(2817, 224, 224, 3) (2817,)


In [19]:
# change labels to one_hot encoding 
amazon_training_labels_one_hot = np.zeros((n_data, n_classes))
# IMPORTANT: class 1 correspond to index 0, class 2 correspond to index 1 
amazon_training_labels_one_hot[np.arange(n_data), amazon_training_labels - 1] = 1
print (amazon_training_labels_one_hot.shape)

(2817, 31)


In [11]:
# fine-tune the Inception V3 model
base_model = InceptionV3(weights = 'imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output 
x = GlobalAveragePooling2D()(x)

# add a fully-connected layer
x = Dense(1024, activation = 'relu')(x)

# add a logistic layer -- we have 31 classes
predictions = Dense(n_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input = base_model.input, outputs = predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

87916544/87910968 [==============================] - 44s 1us/step


D:\Program Files\Python35\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  from ipykernel import kernelapp as app


In [20]:
# train the model on the new data for a few epochs
model.fit(x = amazon_training_data, y=amazon_training_labels_one_hot, epochs = 2)

Epoch 1/2
2817/2817 [==============================] - 991s 352ms/step - loss: 2.3611
Epoch 2/2
2817/2817 [==============================] - 932s 331ms/step - loss: 1.2084


In [21]:
# after training the top layers, freeze the bottom N layers and train the remaining top layers 
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [23]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [24]:
model.fit(x = amazon_training_data, y=amazon_training_labels_one_hot, epochs = 10)

Epoch 1/10
2817/2817 [==============================] - 1159s 412ms/step - loss: 2.7450
Epoch 2/10
2817/2817 [==============================] - 1114s 395ms/step - loss: 0.6269
Epoch 3/10
2817/2817 [==============================] - 1113s 395ms/step - loss: 0.5689
Epoch 4/10
2817/2817 [==============================] - 1112s 395ms/step - loss: 0.5487
Epoch 5/10
2817/2817 [==============================] - 1137s 404ms/step - loss: 0.5075
Epoch 6/10
2817/2817 [==============================] - 1121s 398ms/step - loss: 0.4859
Epoch 7/10
2817/2817 [==============================] - 1124s 399ms/step - loss: 0.4552
Epoch 8/10
2817/2817 [==============================] - 1124s 399ms/step - loss: 0.4348
Epoch 9/10
2817/2817 [==============================] - 1123s 399ms/step - loss: 0.4273
Epoch 10/10
2817/2817 [==============================] - 1121s 398ms/step - loss: 0.3836


In [28]:
# save weights to file
model.save('.\\InceptionV3\\model.h5')
# Future usage: model.load_weights('.\\InceptionV3\\model.h5', by_name=False)